<a href="https://colab.research.google.com/github/katarinagresova/M7DataSP_2020/blob/erik/assignment_07/PoetryCharByChar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

This notebook is inspired by [Keras example](https://keras.io/examples/generative/lstm_character_level_text_generation/) on generating text from Nietzsche's writings with a character-level LSTM. 


## Setup


In [2]:
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import random
import io


## Prepare the data


In [3]:
path = keras.utils.get_file(
    "emily-together.txt", origin="https://raw.githubusercontent.com/katarinagresova/M7DataSP_2020/main/assignment_07/data/emily-together.txt"
)
with io.open(path, encoding="utf-8") as f:
    text = f.read().lower()
print("Corpus length:", len(text))

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 20
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1



188416/184901 [==============================] - 0s 0us/step
Corpus length: 183924
Total chars: 42
Number of sequences: 61302


## Build the model: two LSTM layers


In [5]:
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128, return_sequences=True),
        layers.LSTM(128, return_sequences=False),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)


## Prepare the text sampling function


In [6]:

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)



## Train the model


In [10]:
epochs = 40
batch_size = 128

model.fit(x, y, batch_size=batch_size, epochs=epochs)

Epoch 1/40
479/479 [==============================] - 3s 7ms/step - loss: 0.7627
Epoch 2/40
479/479 [==============================] - 3s 7ms/step - loss: 0.7484
Epoch 3/40
479/479 [==============================] - 3s 7ms/step - loss: 0.7400
Epoch 4/40
479/479 [==============================] - 3s 7ms/step - loss: 0.7255
Epoch 5/40
479/479 [==============================] - 3s 7ms/step - loss: 0.7175
Epoch 6/40
479/479 [==============================] - 3s 7ms/step - loss: 0.7040
Epoch 7/40
479/479 [==============================] - 3s 7ms/step - loss: 0.6951
Epoch 8/40
479/479 [==============================] - 3s 7ms/step - loss: 0.6908
Epoch 9/40
479/479 [==============================] - 3s 7ms/step - loss: 0.6785
Epoch 10/40
479/479 [==============================] - 3s 7ms/step - loss: 0.6716
Epoch 11/40
479/479 [==============================] - 3s 7ms/step - loss: 0.6564
Epoch 12/40
479/479 [==============================] - 3s 7ms/step - loss: 0.6448
Epoch 13/40
479/479 [====

In [11]:
for diversity in [0.2, 0.5, 1.0, 1.2]:
  print("...Diversity:", diversity)

  generated = ""
  sentence = 'sunshine'
  pad_len = maxlen - len(sentence)
  if pad_len < 0:
    sentence = sentence[:maxlen]
  elif pad_len > 0:
      sentence = " " * (pad_len - 1) + sentence + "."

  print('...Generating with seed: "' + sentence + '"')

  for i in range(200):
      x_pred = np.zeros((1, maxlen, len(chars)))
      for t, char in enumerate(sentence):
          x_pred[0, t, char_indices[char]] = 1.0
      preds = model.predict(x_pred, verbose=0)[0]
      next_index = sample(preds, diversity)
      next_char = indices_char[next_index]
      sentence = sentence[1:] + next_char
      generated += next_char



  print("...Generated: ", generated)
  print()

...Diversity: 0.2
...Generating with seed: "           sunshine."


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """


...Generated:  
the but they wearing, by.
whut that feect their wired
and strugg
a dack severetion.
next to the cracket,
and orter of a not can dee,
and lote is not be summer.
the lusction's a wand.
a bird not had t

...Diversity: 0.5
...Generating with seed: "           sunshine."
...Generated:  
not pain reverwention
an ecstas of the dawn
belove the edeal suy,
not have i steal of pluse
oh, first to gate,
countercurious a gold
the lady was two lost,
we could come suffer door,
and prade the ta

...Diversity: 1.0
...Generating with seed: "           sunshine."
...Generated:  
not pains unorchan,
or first the leaves glady.
hout loads flut betted like a wong,
or did how many charitly
is never kistle coucress be
where the meek, swiets upon thy will
with untice should de'seld

...Diversity: 1.2
...Generating with seed: "           sunshine."
...Generated:  
mort, nort thas i orchard,
is dung the reall at the sun;
the stuns that frice,
pease as every time,
to hod his cellours out.
thourst ro